In [1]:
import os
import csv
import numpy as np 
import pandas as pd
from numba import jit, cuda 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer

In [2]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles 

In [16]:
ps_path = r"C:\Users\strut\Desktop\Hons Dataset\Clean"
obfuscated_ps_path = r"C:\Users\strut\Desktop\Hons Dataset\Obfuscated"

file_types_and_labels = [(ps_path, 0), (obfuscated_ps_path, 1)]
print(file_types_and_labels)

[('C:\\Users\\strut\\Desktop\\Hons Dataset\\PS', 0), ('C:\\Users\\strut\\Desktop\\Hons Dataset\\OBF', 1)]


In [17]:
from __future__ import division
import math


def listof(dirName):
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
    return listOfFiles

def hist(source):
    hist = {}
    l = 0
    for e in source:
        l += 1
        if e not in hist:
            hist[e] = 0
        hist[e] += 1
    return (l, hist)

def entropy(hist, l):
    elist = []
    for v in hist.values():
        c = v / l
        elist.append(-c * math.log(c, 2))
    return sum(elist)

def printHist(h):
    def flip(kv):
        return (-kv[1], kv[0])

    h = sorted(h.iteritems(), key=flip)
    print('Sym\thi\tfi\tInf')
    for (k, v) in h:
        print('%s\t%f\t%f\t%f' % (k, v, v / l, -math.log(v / l, 2)))


#############################################################################################################

def Check_Vow(string):
    vowels = "aAeEeIiOoUu"
    final_Vow = len([each for each in string if each in vowels])
    return final_Vow

def Check_Con(string):
    vowels = "bBcCdDfFgGhHjJkKlLmMnNpPqQrRsStTvVwWxXyYzZ"
    final_Con = len([each for each in string if each in vowels])
    return final_Con

def Check_Lines(string):
    count = 1
    with open(file_path, 'r') as f:
        for line in f:
            count += 1
        final_Line = count
        return final_Line

In [18]:
total = 0
tL = 0
tVow = 0
tCon = 0
tSpec = 0
tVCON = 0
tCVOW = 0
tEntropy = 0
tLine_Val = 0

ototal = 0
otL = 0
otVow = 0
otCon = 0
otSpec = 0
otVCON = 0
otCVOW = 0
otEntropy = 0
otLine_Val = 0

for files_path, label in file_types_and_labels:
    for file in listof(files_path):
        file_path = file
        try:
            with open(file_path, "r") as myfile:
                data = myfile.read().replace("\n", "")
                data = str(data)
                ###################################################################################################

                (l, h) = hist(data)
                entrop = entropy(h, l)
                vow_Val = Check_Vow(data)
                con_Val = Check_Con(data)
                line_Val = Check_Lines(file_path)

                if label == 0:
                    total = total + 1
                    tL = tL + l
                    tVow = tVow + vow_Val
                    tCon = tCon + con_Val
                    tSpec = tSpec + (l - (vow_Val + con_Val))
                    tEntropy = tEntropy + entrop
                    tVCON = tVCON + (vow_Val / con_Val)
                    tCVOW = tCVOW  + (con_Val / vow_Val)
                    tLine_Val = tLine_Val + line_Val

                else:
                    ototal = ototal + 1
                    otL = otL + l
                    otVow = otVow + vow_Val
                    otCon = otCon + con_Val
                    otSpec = otSpec + (l - (vow_Val + con_Val))
                    otEntropy = otEntropy + entrop
                    otVCON = otVCON + (vow_Val / con_Val)
                    otCVOW = otCVOW  + (con_Val / vow_Val)
                    otLine_Val = otLine_Val + line_Val


###################################################################################################

        except:
            pass

# with open('Extractions\VIS.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow([
#         "File state", "Total Files", "Total Length", "AVG Length", "Total Vowels",
#         "Vowel Distribution", "Total Consonant", "Consonant Distribution",
#         "Total Special", "Special Distribution", "AVG Entropy", "AVG line"
#     ])
#     writer.writerow([
#         "Normal", total, tL, tL/total, tVow, tVow / tL, tCon, tCon / tL, tSpec,
#         tSpec / tL, tEntropy / total, tLine_Val / total
#     ])
#     writer.writerow([
#         "Obfusscated", ototal, otL, otL/ototal, otVow, otVow / otL, otCon, otCon / otL,
#         otSpec, otSpec / otL, otEntropy / ototal, otLine_Val / ototal
#     ])

with open('Extractions\VIS.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([
        "File state", "Total Files", "AVG Length", "AVG Entropy", "AVG Vowel", 
        "AVG Consonant", "AVG Special", "AVG VCon", "AVG CVow", "AVG Line"
    ])
    
    writer.writerow([
        "Normal", 
        total, 
        
        tL / total, 
        tEntropy / total, 
        tVow / total,
        tCon / total,
        tSpec / total,
        tVCON / total,
        tCVOW / total,
        tLine_Val / total        
    ])

    writer.writerow([
        "Obfusscated", 
        ototal, 
        
        otL / ototal, 
        otEntropy / ototal, 
        otVow / ototal,
        otCon / ototal,
        otSpec / ototal,
        otVCON / ototal,
        otCVOW / ototal,
        otLine_Val / ototal        
    ])




df = pd.read_csv(
    "Extractions\VIS.csv"
)
df

,File state,Total Files,AVG Length,AVG Entropy,AVG Vowel,AVG Consonant,AVG Special,AVG VCon,AVG CVow,AVG Line
0,Normal,6930,6776.717027,4.686726,1445.087302,2518.330014,2813.299711,0.575480,1.765724,193.123377
1,Obfusscated,3406,84134.196124,3.251402,5014.598649,8536.717264,70582.880211,0.505273,7.624260,777.377863


In [19]:
total = 0
tL = 0
tVow = 0
tCon = 0
tSpec = 0
tEntropy = 0
tLine_Val = 0

ototal = 0
otL = 0
otVow = 0
otCon = 0
otSpec = 0
otEntropy = 0
otLine_Val = 0

# with open('Extractions\main extract feature.csv', 'w', newline='') as file_csv:
#     writer = csv.writer(file_csv)
#     writer.writerow([
#         "File", "Type", "String Len", "Entropy", "Vowel", "Consonant",
#         "Special", "V Con Val", "Total Lines"
#     ])

with open('Extractions\VIS.csv', 'w', newline='') as test_csv:
    writer = csv.writer(test_csv)
    writer.writerow([
        "Length", "Entropy", "Vowel", "Consonant", "Special", "V Con",
        "C Vow", "Lines", "State"
    ])

for files_path, label in file_types_and_labels:
    for file in listof(files_path):
        file_path = file
        try:
            with open(file_path, "r") as myfile:
                data = myfile.read().replace("\n", "")
                data = str(data)
                ###################################################################################################

                (l, h) = hist(data)
                entrop = entropy(h, l)
                vow_Val = Check_Vow(data)
                con_Val = Check_Con(data)
                line_Val = Check_Lines(file_path)
                loc = file                    
                     
                    
                with open('Extractions\VIS.csv', 'a', newline='') as test_csv:
                    writer = csv.writer(test_csv)
                    writer.writerow([
                        l, entrop, vow_Val, con_Val,
                        l - (vow_Val + con_Val), vow_Val / con_Val, con_Val / vow_Val, line_Val,
                        label
                    ])
                    
#                 with open('Extractions\main extract feature.data', 'a', newline='') as test_csv:
#                     writer = csv.writer(test_csv)
#                     writer.writerow([
#                         label, l, entrop, vow_Val, con_Val,
#                         l - (vow_Val + con_Val), vow_Val / con_Val, con_Val / vow_Val, line_Val
#                     ])

###################################################################################################

        except:
            pass

df = pd.read_csv(
    "Extractions\VIS.csv"
) 
df

,Length,Entropy,Vowel,Consonant,Special,V Con,C Vow,Lines,State
0,3610,4.861588,805,1390,1415,0.579137,1.726708,107,0
1,1012,4.750210,225,414,373,0.543478,1.840000,36,0
2,712,4.891841,219,363,130,0.603306,1.657534,18,0
3,212,3.869694,19,29,164,0.655172,1.526316,2,0
4,212,3.869694,19,29,164,0.655172,1.526316,2,0
...,...,...,...,...,...,...,...,...,...
10227,5704,2.650124,26,54,5624,0.481481,2.076923,4,1
10228,2918,3.239057,67,220,2631,0.304545,3.283582,4,1
10229,1131,3.489889,167,291,673,0.573883,1.742515,17,1
10230,6508,1.683901,27,68,6413,0.397059,2.518519,4,1
